# Pricing de CDS 

In [55]:
import pandas as pd
import math
import numpy as np

### Parametres du CDS

In [ ]:
emetteurs = 'Atos.SA'
maturite = 5 
taux_recouvrement = 0.4  
taux_actualisation = 0.05  
proba_defaut = 0.02


## 1) Valeur actuelle esperée des paiments sur le CDS ( paiement s par an)

In [40]:


class prime_vendeur_CDS:
    
    def __init__(self, proba_defaut, taux_actualisation, maturite):
        self.df = pd.DataFrame()
        self.proba_defaut = proba_defaut
        self.taux_actualisation = taux_actualisation
        self.maturite = maturite

    def calculer_valeur(self):
        self.df['Maturité'] = range(1, self.maturite + 1)
        self.df['Probabilité de survie'] = -self.proba_defaut * self.df['Maturité']
        self.df['Probabilité de survie'] = self.df['Probabilité de survie'].apply(lambda x: math.exp(x))
        self.df['Probabilité de defaut'] = abs(self.df['Probabilité de survie'] - self.df['Probabilité de survie'].shift(1))
        self.df.iloc[0, self.df.columns.get_loc('Probabilité de defaut')] = self.proba_defaut

        self.df['Actualisation'] = -self.taux_actualisation * self.df['Maturité']
        self.df['Actualisation'] = self.df['Actualisation'].apply(lambda x: math.exp(x))
        self.df['Valeur actuelle'] = self.df['Actualisation'] * self.df['Probabilité de survie']
        
    def probabilité_defaut(self):
        return (self.df['Probabilité de defaut'])
            
    def proba_survie(self):
        return (self.df['Probabilité de survie'])

    def afficher_resultat(self):
        print(f"La valeur actuelle esperée de l'ensemble des paiements est égal à {round(sum(self.df['Valeur actuelle']),4)} s")
        print(f"pour le vendeur du CDS")

        return (self.df)
    
    def valeur_espérée(self):
        return sum(self.df['Valeur actuelle'])

    



In [45]:
prime_vendeur =prime_vendeur_CDS(proba_defaut, taux_actualisation, maturite)
prime_vendeur.calculer_valeur()
prime_vendeur.probabilité_defaut()
prime_vendeur.afficher_resultat()


La valeur actuelle esperée de l'ensemble des paiements est égal à 4.0728 s
pour le vendeur du CDS


,Maturité,Probabilité de survie,Probabilité de defaut,Actualisation,Valeur actuelle
0,1,0.980199,0.020000,0.951229,0.932394
1,2,0.960789,0.019409,0.904837,0.869358
2,3,0.941765,0.019025,0.860708,0.810584
3,4,0.923116,0.018648,0.818731,0.755784
4,5,0.904837,0.018279,0.778801,0.704688


## 2) Valeur actuelle esperée des paiments sur le CDS en cas de défaut ( pour une unité de nominal )

In [ ]:
class prime_acheteur_CDS(prime_vendeur_CDS):
    
    def __init__(self, taux_recouvrement, taux_actualisation, proba_defaut, maturite):
        super().__init__(proba_defaut, taux_actualisation, maturite)
        self.df1 = pd.DataFrame()
        self.taux_recouvrement = taux_recouvrement
        self.prime_vendeur = prime_vendeur

    def calculer_recouvrement(self):
        
        self.df1['Date'] = np.arange(0.5, self.maturite+ 0.5, 1)
        self.df1['Probabilité de defaut'] = prime_vendeur.probabilité_defaut()
        self.df1['Taux de recouvrement'] = self.taux_recouvrement
        self.df1['Actualisation'] = -self.taux_actualisation * self.df1['Date']
        self.df1['Actualisation'] = self.df1['Actualisation'].apply(lambda x: math.exp(x))
        self.df1['Valeur actuelle'] = self.df1['Actualisation'] * self.df1['Probabilité de defaut'] * (1 - self.taux_recouvrement)

    def afficher_resultat(self):
        print(f"La valeur actuelle esperée du montant total reçu (payoff)")
        print(f"en cas de défaut est {round(sum(self.df1['Valeur actuelle']),4)}")

        return self.df1
        
    def valeur_espérée(self):
        return sum(self.df1['Valeur actuelle'])


In [46]:

prime_acheteur = prime_acheteur_CDS(taux_recouvrement,proba_defaut, taux_actualisation, maturite)
prime_acheteur.calculer_recouvrement()
prime_acheteur.afficher_resultat()

La valeur actuelle esperée du montant total reçu (payoff)
en cas de défaut est 0.0545


,Date,Probabilité de defaut,Taux de recouvrement,Actualisation,Valeur actuelle
0,0.5,0.020000,0.4,0.990050,0.011881
1,1.5,0.019409,0.4,0.970446,0.011301
2,2.5,0.019025,0.4,0.951229,0.010858
3,3.5,0.018648,0.4,0.932394,0.010432
4,4.5,0.018279,0.4,0.913931,0.010023


## 3) Valeur actuelle residuel esperée des paiments sur le CDS en cas de défaut ( pour une unité de nominal )

In [ ]:
class prime_residuel_vendeur_CDS(prime_vendeur_CDS):
    

    def __init__(self, taux_actualisation, proba_defaut, maturite):
        super().__init__(proba_defaut, taux_actualisation, maturite)
        self.df1 = pd.DataFrame()
        

    def calculer_recouvrement(self):
        
        self.df1['Date'] = np.arange(0.5, self.maturite+ 0.5, 1)
        self.df1['Probabilité de defaut'] = prime_vendeur.probabilité_defaut()
        self.df1['Actualisation'] = -self.taux_actualisation * self.df1['Date']
        self.df1['Actualisation'] = self.df1['Actualisation'].apply(lambda x: math.exp(x))
        self.df1['Valeur actuelle'] = self.df1['Actualisation'] * self.df1['Probabilité de defaut'] * 0.5

    def afficher_resultat(self):
        print(f"La valeur actuelle esperée du montant residuel reçu (payoff)")
        print(f"en cas de défaut est {round(sum(self.df1['Valeur actuelle']),4)} s")
        return self.df1
        
    def valeur_espérée(self):
        return sum(self.df1['Valeur actuelle'])

In [47]:
spread_residuel = prime_residuel_vendeur_CDS(taux_actualisation,proba_defaut, maturite)
spread_residuel.calculer_recouvrement()
spread_residuel.afficher_resultat()

La valeur actuelle esperée du montant residuel reçu (payoff)
en cas de défaut est 0.0423 s


,Date,Probabilité de defaut,Actualisation,Valeur actuelle
0,0.5,0.020000,0.975310,0.009753
1,1.5,0.019409,0.927743,0.009003
2,2.5,0.019025,0.882497,0.008395
3,3.5,0.018648,0.839457,0.007827
4,4.5,0.018279,0.798516,0.007298


## 4) Evaluation du CDS 

In [54]:
print(f"Le Spread du CDS sur {emetteurs} est de :")
print(f"{round((prime_acheteur.valeur_espérée()/(prime_vendeur.valeur_espérée() + spread_residuel.valeur_espérée())*10000),2)} bp par an")

Le Spread du CDS sur Atos.SA est de :
132.43 bp par an
